

# 结巴分词






三种分词模式 ： 
A、精确模式：试图将句子最精确地切开，适合文本分析。默认是精确模式。 
B、全模式：把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义
C、搜索引擎模式：在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词
 注：当指定jieba.cut的参数HMM=True时，就有了新词发现的能力。






结巴分词算法：https://blog.csdn.net/rav009/article/details/12196623


In [1]:
import jieba
'''
cut方法有两个参数
1)第一个参数是我们想分词的字符串
2)第二个参数cut_all是用来控制是否采用全模式
'''
#全模式
word_list = jieba.cut("今天天气真好。亲爱的，我们去远足吧！",cut_all=True)
print( "全模式：","|".join(word_list))
#精确模式 , 默认就是精确模式
word_list = jieba.cut("今天天气真好。亲爱的，我们去远足吧！",cut_all=False)
print ("精确模式：","|".join(word_list))
#搜索引擎模式
word_list = jieba.cut_for_search("今天天气真好。亲爱的，我们去远足吧！")
print ("搜索引擎：","|".join(word_list))


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\haoyu\AppData\Local\Temp\jieba.cache
Loading model cost 0.995 seconds.
Prefix dict has been built succesfully.


全模式： 今天|今天天气|天天|天气|真好|||亲爱|的|||我们|去|远足|吧||
精确模式： 今天天气|真|好|。|亲爱|的|，|我们|去|远足|吧|！
搜索引擎： 今天|天天|天气|今天天气|真|好|。|亲爱|的|，|我们|去|远足|吧|！


In [ ]:
import jieba
def read_file(filename):
        stop = [line.strip() for line in  open('F:/课程设计及上机实验/python/文本分析/中文分词/Chinese-StopWords.txt','r',encoding='utf-8').readlines()]  #停用词
        f = open(filename,'r',encoding='utf-8')
        line = f.readline()
        str = []
        while line:
            s = line.split('\t')
            fenci = jieba.cut(s[0],cut_all=False)  #False默认值：精准模式
            #word=print( "|".join(fenci))
            str.append(list(set(fenci)-set(stop)))
            line = f.readline()
        return str


In [ ]:
stop = [line.strip() for line in  open('F:/课程设计及上机实验/python/文本分析/中文分词/Chinese-StopWords.txt','r',encoding='utf-8').readlines()]  #停用词
f = open('F:/课程设计及上机实验/python/文本分析/中文分词/test/test1.txt','r',encoding='utf-8')
line = f.readline()
str = []
while line:
    s = line.split('\t')#\t\r\n均为转义字符，\t是横向跳到下一个制表符位置
    fenci = jieba.cut(s[0],cut_all=False)  #False默认值：精准模式
    str.append(list(set(fenci)-set(stop)))
    line = f.readline()
print(str)


In [ ]:
from nltk.probability import  FreqDist,ConditionalFreqDist
from nltk.metrics import  BigramAssocMeasures


获取信息量最高(前number个)的特征(卡方统计)
nltk中的FreqDist,ConditionalFreqDist：https://blog.csdn.net/zhuzuwei/article/details/79154371 
条件频率分布需要处理的是配对列表，每对的形式是（条件，事件），conditions()函数会返回这里的条件。

In [ ]:
def jieba_feature(number): 
        posWords = []
        negWords = []
        for items in read_file('F:/课程设计及上机实验/python/文本分析/中文分词/clothing/pos.txt'):
            #把集合的集合变成集合
            for item in items:
                posWords.append(item)
        for items in read_file('F:/课程设计及上机实验/python/文本分析/中文分词/clothing/neg.txt'):
            for item in items:
                negWords.append(item)

        word_fd = FreqDist() #可统计所有词的词频
        cond_word_fd = ConditionalFreqDist() #可统计积极文本中的词频和消极文本中的词频

        for word in posWords:
            word_fd[word] += 1
            cond_word_fd['pos'][word] += 1

        for word in negWords:
            word_fd[word] += 1
            cond_word_fd['neg'][word] += 1

        pos_word_count = cond_word_fd['pos'].N() #积极词的数量
        neg_word_count = cond_word_fd['neg'].N() #消极词的数量
        total_word_count = pos_word_count + neg_word_count

        word_scores = {}#包括了每个词和这个词的信息量

        for word, freq in word_fd.items():
            pos_score = BigramAssocMeasures.chi_sq(cond_word_fd['pos'][word],
                       (freq, pos_word_count),total_word_count) 
            #计算积极词的卡方统计量，这里也可以计算互信息等其它统计量
            neg_score = BigramAssocMeasures.chi_sq(cond_word_fd['neg'][word],
                        (freq, neg_word_count), total_word_count) 
            word_scores[word] = pos_score + neg_score 
            #一个词的信息量等于积极卡方统计量加上消极卡方统计量

        best_vals = sorted(word_scores.items(), key=lambda item:item[1], reverse=True)[:number] #把词按信息量倒序排序。number是特征的维度，是可以不断调整直至最优的
        best_words = set([w for w,s in best_vals])
        return dict([(word, True) for word in best_words])  


keys()详解：https://www.cnblogs.com/wushuaishuai/p/7738313.html

In [ ]:
def build_features():
     #feature = bag_of_words(text())#第一种：单个词
     #feature = bigram(text(),score_fn=BigramAssocMeasures.chi_sq,n=500)#第二种：双词
     #feature =  bigram_words(text(),score_fn=BigramAssocMeasures.chi_sq,n=500)#第三种：单个词和双个词
        feature = jieba_feature(1000)#第四种：结巴分词

        posFeatures = []
        for items in read_file('F:/课程设计及上机实验/python/文本分析/中文分词/clothing/pos.txt'):
            a = {}
            for item in items:
                if item in feature.keys():
                    a[item]='True'
            posWords = [a,'pos'] #为积极文本赋予"pos"
            posFeatures.append(posWords)
        negFeatures = []
        for items in read_file('F:/课程设计及上机实验/python/文本分析/中文分词/clothing/neg.txt'):
            a = {}
            for item in items:
                if item in feature.keys():
                    a[item]='True'
            negWords = [a,'neg'] #为消极文本赋予"neg"
            negFeatures.append(negWords)
        return posFeatures,negFeatures


In [ ]:
#获得训练数据
posFeatures,negFeatures =  build_features()
posFeatures

In [ ]:
from random import shuffle
shuffle(posFeatures) 
shuffle(negFeatures) #把文本的排列随机化 
#N1=int(0.3*total_word_count/2)
train =  posFeatures[300:]+negFeatures[300:]#训练集(70%)
test = posFeatures[:300]+negFeatures[:300]#验证集(30%)
data,tag = zip(*test)#分离测试集合的数据和标签，便于验证和测试

In [ ]:
print(data,tag)